In [1]:
import pandas as pd
from sklearn.metrics import classification_report
import pickle

In [2]:
# 1. Load and preprocess the second‐order dataset
df2 = pd.read_csv("/kaggle/input/ethereum-transactions-for-fraud-detection/second_order_df.csv")
df2.columns = df2.columns.str.strip()

# Drop any stray index column
if 'Unnamed: 0' in df2.columns:
    df2.drop('Unnamed: 0', axis=1, inplace=True)

# Convert Unix timestamp and extract time features
df2['TimeStamp'] = pd.to_datetime(df2['TimeStamp'], unit='s')
df2['hour']  = df2['TimeStamp'].dt.hour
df2['day']   = df2['TimeStamp'].dt.day
df2['month'] = df2['TimeStamp'].dt.month
df2['year']  = df2['TimeStamp'].dt.year

In [3]:
# 2. Prepare features & target
features = ['BlockHeight', 'Value', 'hour', 'day', 'month', 'year', 'From', 'To']
X2 = df2[features]
y2 = df2['isError']

In [4]:
# 3. Load all your saved pipelines
model_keys = ["RF", "MLP", "XGB", "RF+MLP", "RF+XGB", "MLP+XGB", "RF+MLP+XGB"]
pipelines = {}
for key in model_keys:
    fname = f"/kaggle/input/comp517_models/scikitlearn/default/1/pipeline_{key.replace('+','_')}.pkl"
    with open(fname, "rb") as f:
        pipelines[key] = pickle.load(f)

In [5]:
# 4. Evaluate each model on the second‐order set and summarize correctness
summary = []
total = len(y2)

for name, pipe in pipelines.items():
    y_pred = pipe.predict(X2)

    # Counts
    valid_count = int((y_pred == 0).sum())
    error_count = int((y_pred == 1).sum())
    correct    = int((y_pred == y2).sum())
    accuracy   = correct / total

    # Correctness within predicted-valid
    correct_valid = int(((y_pred == 0) & (y2 == 0)).sum())
    valid_correct_pct = correct_valid / valid_count if valid_count > 0 else 0.0

    # Correctness within predicted-error
    correct_error = int(((y_pred == 1) & (y2 == 1)).sum())
    error_correct_pct = correct_error / error_count if error_count > 0 else 0.0

    summary.append({
        "Model":            name,
        "Valid (0)":        valid_count,
        "Error (1)":        error_count,
        "Correct":          correct,
        "Total":            total,
        "Accuracy":         round(accuracy, 6),
        "Valid_Correct_%":  round(valid_correct_pct, 6),
        "Error_Correct_%":  round(error_correct_pct, 6)
    })

summary_df = pd.DataFrame(summary)
print("Overall Summary with Class-wise Correctness %:")
print(summary_df)

Overall Summary with Class-wise Correctness %:
        Model  Valid (0)  Error (1)  Correct    Total  Accuracy  \
0          RF    6792684       1837  6463903  6794521  0.951340   
1         MLP    6792901       1620  6461174  6794521  0.950939   
2         XGB    6794394        127  6462903  6794521  0.951193   
3      RF+MLP    6793626        895  6462459  6794521  0.951128   
4      RF+XGB    6794203        318  6463102  6794521  0.951223   
5     MLP+XGB    6792882       1639  6461221  6794521  0.950946   
6  RF+MLP+XGB    6794397        124  6462908  6794521  0.951194   

   Valid_Correct_%  Error_Correct_%  
0         0.951380         0.804573  
1         0.951165         0.003086  
2         0.951193         0.968504  
3         0.951211         0.318436  
4         0.951220         1.000000  
5         0.951170         0.023185  
6         0.951193         1.000000  
